In [1]:
import osmnx as ox
import pandas as pd
from test_coord_copy import coord
from weights import weights
from aux_functions.distance_func import euclidan_dist , haversine_distance

In [4]:
clients=coord #coordenadas de los clients y el deposito
distancia_clients_distancia=[]
distancia_clients_tiempo=[]


for i in range (len(clients)):
    orig = clients[i] #primer nodo
    for j in range (len(clients)): 
        dest = clients[j] #nodo a visitar
        orig_node = ox.distance.nearest_nodes(G, orig[1], orig[0]) #ubicar el nodo de origen en el grafo de OSMnx
        destination_node = ox.distance.nearest_nodes(G, dest[1], dest[0]) #ubicar el nodo de destino en el grafo de OSMnx
        route1 = ox.shortest_path(G, orig_node, destination_node, weight="length") #calcular la ruta mas corta
        distancia_clients_distancia.append(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))  #obtener atributo de distancia de la ruta
        distancia_clients_tiempo.append(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time"))) #obtener atributo de tiempo de la ruta

C:\Users\JUAN SEBASTIAN\AppData\Local\Temp\ipykernel_11244\399016743.py:13: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `route_to_gdf` function instead.
  distancia_clients_distancia.append(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))  #obtener atributo de distancia de la ruta
C:\Users\JUAN SEBASTIAN\AppData\Local\Temp\ipykernel_11244\399016743.py:14: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `route_to_gdf` function instead.
  distancia_clients_tiempo.append(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time"))) #obtener atributo de tiempo de la ruta
C:\Users\JUAN SEBASTIAN\AppData\Local\Temp\ipykernel_11244\399016743.py:13: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `route_to_gdf` function instead

# IMPORTING DATASETS

## IMPORTING WEIGHTS DATAFRAME

In [2]:
weights = weights
weights

[25150.0,
 19615.0,
 39310.0,
 20860.0,
 35730.0,
 23340.0,
 17630.0,
 60797.0,
 21705.0,
 49475.0,
 38970.0,
 7325.0,
 3855.0,
 49380.0,
 19807.0,
 20610.0,
 27590.0,
 149881.0,
 14140.0,
 26260.0]

In [17]:
weights_df = pd.DataFrame(weights, columns=['weight']) #Se puede renombrar la columna
weights_df

,weight
0,25150.0
1,19615.0
2,39310.0
3,20860.0
4,35730.0
5,23340.0
6,17630.0
7,60797.0
8,21705.0
9,49475.0


In [41]:
4.978074 + 4.763573 - 0.214513

9.527134

## IMPORTING DEMAND DATAFRAME

In [4]:
demand = [25150,
19615,
39310,
20860,
35730,
23340,
17630,
60797,
21705,
49475,
38970,
7325,
3855,
49380,
19807,
20610,
27590,
149881,
14140,
26260,
50570,
32825,
31990,
7280,
26260,
4430,
29500,
21745,
32975,
14965,
5986,
41415,
75053,
40120,
34013,
26260,
110118,
18655,
8505,
21510,
115980,
30720,
68270,
28995,
50700,
43085,
14595,
37235,
25940,
75850,
33449,
10336,
19095,
25770,
24720,
37875,
41599,
8375,
9060,
13010,
17595,
21294,
10204,
17185,
11175,
152910,
32146,
68742,
12981,
115485,
12890,
25250,
28231,
15895,
32105,
44285,
4320,
20500,
50100,
41730,
6430,
75840,
25450,
19200,
26260,
8980,
6310,
87048,
67100,
10555,
49338,
44040,
62390,
31175,
3580,
35815,
19775,
6383,
12920,
17980,
7850,
16600,
39155,
6755,
25550,
33350,
37765,
10822,
19570,
34814,
39900,
23840,
38850,
67990,
41690,
28990,
7961,
34000,
24745,
28644,
7050,
11175,
36005,
3400,
27925,
10190,
37920,
26250,
20275,
22810,
4660,
7780,
19880,
70645,
117085,
31585,
14000,
137735,
36565,
18117,
28680,
41655,
4545,
12187,
5475,
25764,
37920,
17150,
15713,
44235,
53460,
21800,
15895,
36000,
50100,
15375,
37886,
880,
23270,
27605,
38330,
27480,
21595,
53175,
11000,
6390,
22635,
20510,
6580,
34405,
26555,
3820,
17330,
6060,
71681,
26260,
70525,
10110,
140579,
35115,
136196,
21485,
34860,
4530,
31740,
17318,
29746,
16172,
42772,
92082,
21851,
13017,
17009,
49462,
50100,
8020,
11295,
36675,
40270,
13925,
12625,
61101,
20420,
18165,
2628,2,
13630,
6805,
14060,
4260
]

## IMPORTING CAPACITY

In [5]:
capacity_vehicle = 450000
number_vehicles = 5

# SAVINGS MATRIX

* Se deben calcular los ahorros de la siguiente forma:
* Sij = Ci0 + C0j - Cij para todo i distinto de j

In [6]:
def calculate_savings(distance_type):
    if distance_type == 'haversine':
        df_distance_matrix = haversine_distance(coord=coord)
    else:
        df_distance_matrix = euclidan_dist(coord=coord)
    savings_dict = {}
    for i in df_distance_matrix.index:
        for j in df_distance_matrix.columns:
            if i != j and i != 0 and j!=0:
                a = max(int(i),int(j))
                b = min(int(i),int(j))
                key = '(' + str(a) + ',' + str(b) + ')'
                savings_dict[key] = df_distance_matrix[0][i] + df_distance_matrix[0][j] - df_distance_matrix[i][j]

    # put savings in a pandas dataframe, and sort by descending
    savings_df = pd.DataFrame.from_dict(savings_dict, orient = 'index')
    savings_df.rename(columns = {0:'saving'}, inplace = True)
    savings_df.sort_values(by = ['saving'], ascending = False, inplace = True)

    return savings_df


In [7]:
savings_df = calculate_savings(distance_type='euclidian')
savings_df

,saving
"(13,3)",0.085757
"(13,10)",0.077362
"(10,3)",0.076896
"(19,13)",0.076395
"(19,3)",0.075108
...,...
"(17,3)",0.000032
"(17,8)",0.000030
"(17,11)",0.000025
"(17,10)",0.000004


In [8]:
savings_df_h = calculate_savings(distance_type='haversine')
savings_df_h

,saving
"(13,3)",9.527134
"(13,10)",8.592371
"(10,3)",8.540916
"(19,13)",8.492379
"(19,3)",8.348916
...,...
"(17,3)",0.003503
"(17,8)",0.003283
"(17,11)",0.002808
"(17,10)",0.000489


In [8]:
# Lista para almacenar las claves a mantener
claves_a_mantener = []

# Recorrer las claves del diccionario
for clave in savings.keys():
    # Verificar si el segundo elemento de la clave no es 1
    if clave.split(',')[1][0] != '1':
        # Agregar la clave a la lista de claves a mantener
        claves_a_mantener.append(clave)

# Crear un nuevo diccionario con las claves a mantener
diccionario_filtrado = {clave: savings[clave] for clave in claves_a_mantener}

# Imprimir el diccionario resultante
print(diccionario_filtrado)

IndexError: list index out of range

In [9]:
def get_node(link):
    link = link[1:]
    link = link[:-1]
    nodes = link.split(',')
    return [int(nodes[0]), int(nodes[1])]

In [10]:
new_indexes = savings_df.rename(index=get_node).index
savings_df_new = pd.DataFrame(savings_df.values, index=new_indexes, columns=savings_df.columns)

In [11]:
savings_df_new

,saving
"[13, 3]",0.085757
"[13, 10]",0.077362
"[10, 3]",0.076896
"[19, 13]",0.076395
"[19, 3]",0.075108
...,...
"[17, 3]",0.000032
"[17, 8]",0.000030
"[17, 11]",0.000025
"[17, 10]",0.000004


In [38]:
test_routes = [[12, 4], [1, 8,7]]

In [32]:
test_routes

[12, 4, 1, 8]

In [29]:
saving[1]

7

In [28]:
saving[0]

12

In [41]:
[saving[0],saving[1]]

[12, 7]

In [40]:
set([saving[0],saving[1]])

{7, 12}

In [39]:
all(x in test_routes for x in set([saving[0],saving[1]]))

False

In [31]:
(saving[1] and saving[0]) in test_routes

True

In [44]:
saving[0]

12

In [42]:
savings_df_new.index

Index([ [13, 3], [13, 10],  [10, 3], [19, 13],  [19, 3], [13, 11],  [11, 3],
       [11, 10], [19, 10], [19, 11],
       ...
        [14, 7],   [9, 8],  [17, 6],   [7, 1], [17, 13],  [17, 3],  [17, 8],
       [17, 11], [17, 10],  [12, 7]],
      dtype='object', length=171)

In [43]:
weights_df.head()

,weight
0,25150.0
1,19615.0
2,39310.0
3,20860.0
4,35730.0


In [51]:
#valida la capacidad del vehiculo 
def validate_cap(dem_node_i, dem_node_j, dem_route):
    sum_total = dem_node_i + dem_node_j + dem_route
    return sum_total <= capacity_vehicle

#

In [52]:
[saving[0],saving[1]]

[12, 7]

In [53]:
test_routes

[[12, 4], [1, 8, 7]]

In [101]:
#any(value in route for route in test_routes for value in [saving[0],saving[1]])

route_x =[]

#[route for route in test_routes for value in [saving[0],saving[1]]]

#validacion que asigna la ruta a la cual pertenece uno de los dos nodos si existen,
# si ambos existen devuelve dos rutas si estan en diferentes rutas 
#pdt: si devuelve una ruta, validar que ambos esten ahi, para descartar.
[route_x.append(route) if (value in route and route not in route_x) else route_x.clear 
 for route in test_routes for value in [4,12]]

print(route_x)

x= [4,12]
#validacion en caso de que se devuelva 1 sola lista, validar si ambos numeros estan dentro de ella (True), 
#para descartarlos de las rutas. 
var = True if len(route_x) ==1 and set(x).issubset(set(route_x[0])) else False
print(var)


[[12, 4]]
True


In [ ]:
routes = []
for saving in savings_df_new.index:
    if validate_cap(weights_df['weight'].iloc[saving[0]], weights_df['weight'].iloc[saving[1]], 0):
        try:
            if routes == []:

            else:
                if any(value in route for route in routes for value in [saving[0],saving[1]]):
                    pass
                elif:

            routes.append(saving[0], saving[1])



450000
False


In [32]:
savings_df_new.loc[[any(isinstance(x, int) and x > 18 for x in index) for index in savings_df_new.index]]

,saving
"[19, 13]",0.076395
"[19, 3]",0.075108
"[19, 10]",0.067897
"[19, 11]",0.067605
"[19, 6]",0.062926
"[19, 15]",0.060391
"[19, 16]",0.052634
"[19, 4]",0.037722
"[19, 18]",0.037193
"[19, 2]",0.034939


In [14]:
# determine if a node is interior to a route
def interior(node, route):
    try:
        i = route.index(node)
        # adjacent to depot, not interior
        if i == 0 or i == (len(route) - 1):
            label = False
        else:
            label = True
    except:
        label = False
    
    return label

In [15]:
# merge two routes with a connection link
def merge(route0, route1, link):
    if route0.index(link[0]) != (len(route0) - 1):
        route0.reverse()
    
    if route1.index(link[1]) != 0:
        route1.reverse()
        
    return route0 + route1

In [ ]:
# sum up to obtain the total passengers belonging to a route
def sum_cap(route):
    sum_cap = 0
    for node in route:
        sum_cap += nodes.demand[node]
    return sum_cap

## NOTAS
- Hay que empezar el proceso de iterar para crear las rutas, acá hay que tener en cuenta los pesos (demanda)
- ¿Cómo puedo identificar a qué cliente pertenece el valor de demanda? En el índice [13,3] se identifican los nodos o clientes entonces de allí se pueden tomar.
- Se tiene el archivo weights con las demandas por clientes está desde el cliente 1 pues 0 es CD (Source)
- https://www.youtube.com/watch?v=oKvToy9OMkA&t=1466s&ab_channel=MarcelaVillaMarulanda muy buena explicación